In [1]:
import pandas as pd
import numpy as np

### Backdoor dataset (Optional)

In [3]:
bd = pd.read_csv(r"dataset\UNSW_NB15_traintest_backdoor.csv").to_numpy()
X_in1, X_in2 = [], []
for d in bd:
    if d[-1]==0:
        X_in1.append(d[:-1])
    else:
        X_in2.append(d[:-1])
X_in1, X_in2 = np.array(X_in1), np.array(X_in2)
print(X_in1.shape, X_in2.shape)

93000 2329


### Anti-Frozen Protein dataset (Optional)

In [ ]:
X_in1 = pd.read_csv(r"dataset\ad_AFP11.csv").to_numpy().T[1:].T
X_in2 = pd.read_csv(r"dataset\UNSW_NB15_traintest_backdoor.csv").to_numpy().T[1:].T
print(X_in1.shape, X_in2.shape)

### Classification

In [4]:
from RPCP import RPEnsemble_CP
from Base_classify import base
from Aggregate import AggregatedCP

In [5]:
n_exp = 5
exp_model = "KNN"
alpha = 0.05
base_acc = []
rpecp_acc, rpecp_coverage, rpecp_length = [], [], [] 
icp_acc, icp_coverage, icp_length = [], [], []
ccp_acc, ccp_coverage, ccp_length = [], [], [] 
bcp_acc, bcp_coverage, bcp_length = [], [], [] 

for exp in range(n_exp):
    print("exp:", exp+1, "start:")
    np.random.seed(2024+exp)
    # shuffle_1 = np.random.permutation(np.arange(len(X_in1)))
    # shuffle_2 = np.random.permutation(np.arange(len(X_in2)))
    # X_in1 = X_in1[shuffle_1]
    # X_in1 = X_in1[shuffle_2]
    
    # Backdoor
    X_train = np.vstack((X_in1[exp:500+exp],X_in2[exp:500+exp])),np.concatenate((np.repeat(0,500),np.repeat(1,500)))
    X_test = np.vstack((X_in1[exp:500+exp],X_in2[500+exp:1000+exp])), np.concatenate((np.repeat(0,500),np.repeat(1,500)))

    # Anti-Frozen Protein
    """ X_train = np.vstack((X_in1[exp:200+exp], X_in2[exp:200+exp])),np.concatenate((np.repeat(0,200),np.repeat(1,200)))
    X_test = np.vstack((X_in1[200+exp:400+exp], X_in2[200+exp:400+exp])), np.concatenate((np.repeat(0,200),np.repeat(1,200))) """

    rpecp = RPEnsemble_CP(rp="axis", d = 10, B1=20, B2=50, classifier=exp_model, alpha=alpha)
    rpecp_res = rpecp.RP_Conformal(X_train, X_test)
    rpecp_acc.append(rpecp_res[0])
    rpecp_coverage.append(rpecp_res[1])
    rpecp_length.append(rpecp_res[2])
    print(rpecp_res)
    print("RPECP completed")

    base_classifier = base(classifier = exp_model,random_state=2024+exp)
    base_res = base_classifier.predict(X_train, X_test)
    base_acc.append(base_res)
    print(base_res)
    print("Base completed")

    ICPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="ICP")
    icp_res = ICPredictor.predict(X_train, X_test)
    icp_acc.append(icp_res[0])
    icp_coverage.append(icp_res[1])
    icp_length.append(icp_res[2])
    print("ICP completed")


    CCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="CCP")
    ccp_res = CCPredictor.predict(X_train, X_test)
    ccp_acc.append(ccp_res[0])
    ccp_coverage.append(ccp_res[1])
    ccp_length.append(ccp_res[2])
    print("CCP completed")

    BCPredictor = AggregatedCP(classifier=exp_model, alpha=alpha, mode="BCP")
    bcp_res = BCPredictor.predict(X_train, X_test)
    bcp_acc.append(bcp_res[0])
    bcp_coverage.append(bcp_res[1])
    bcp_length.append(bcp_res[2])
    print("BCP completed")

    print("---------------------------------------\n")

exp: 1 start:
(0.948, 0.959, 1.0312825860271115)
RPECP completed
0.937
Base completed
ICP completed
CCP with 10 folds
CCP completed
BCP completed
---------------------------------------

exp: 2 start:
(0.949, 0.969, 1.0371517027863777)
RPECP completed
0.937
Base completed
ICP completed
CCP with 10 folds
CCP completed
BCP completed
---------------------------------------

exp: 3 start:
(0.95, 0.955, 1.012565445026178)
RPECP completed
0.937
Base completed
ICP completed
CCP with 10 folds
CCP completed
BCP completed
---------------------------------------

exp: 4 start:
(0.95, 0.957, 1.0240334378265412)
RPECP completed
0.937
Base completed
ICP completed
CCP with 10 folds
CCP completed
BCP completed
---------------------------------------

exp: 5 start:
(0.948, 0.95, 1.0157894736842106)
RPECP completed
0.937
Base completed
ICP completed
CCP with 10 folds
CCP completed
BCP completed
---------------------------------------



In [6]:
print("base acc:", np.mean(base_acc), np.std(base_acc))
print("-----------------------------------------------------")
print("RPECP acc:", np.mean(rpecp_acc), np.std(rpecp_acc))
print("RPECP coverage:", np.mean(rpecp_coverage), np.std(rpecp_coverage))
print("RPECP length:", np.mean(rpecp_length), np.std(rpecp_length))
print("-----------------------------------------------------")
print("ICP acc:", np.mean(icp_acc), np.std(icp_acc))
print("ICP coverage:", np.mean(icp_coverage), np.std(icp_coverage))
print("ICP length:", np.mean(icp_length), np.std(icp_length))
print("-----------------------------------------------------")
print("CCP acc:", np.mean(ccp_acc), np.std(ccp_acc))
print("CCP coverage:", np.mean(ccp_coverage), np.std(ccp_coverage))
print("CCP length:", np.mean(ccp_length), np.std(ccp_length))
print("-----------------------------------------------------")
print("BCP acc:", np.mean(bcp_acc), np.std(bcp_acc))
print("BCP coverage:", np.mean(bcp_coverage), np.std(bcp_coverage))
print("BCP length:", np.mean(bcp_length), np.std(bcp_length))

base acc: 0.937 0.0
-----------------------------------------------------
RPECP acc: 0.9489999999999998 0.0008944271909999167
RPECP coverage: 0.958 0.006260990336999417
RPECP length: 1.024164529070084 0.009209027139376363
-----------------------------------------------------
ICP acc: 0.8583999999999999 0.0048414873747640854
ICP coverage: 0.9879999999999999 0.001095445115010333
ICP length: 1.1311680956212136 0.005795123082569617
-----------------------------------------------------
CCP acc: 0.6272 0.036328501207729444
CCP coverage: 1.0 0.0
CCP length: 1.3728000000000002 0.03632850120772944
-----------------------------------------------------
BCP acc: 0.8930000000000001 0.008438009243891603
BCP coverage: 0.9814 0.0062481997407253295
BCP length: 1.0900040114630607 0.013039078614039941
